> # Imports

In [79]:
# standard python library imports
import numpy as np 
import pandas as pd

# importing pyspark library
import pyspark

# importing all pyspark functions
from pyspark.sql.functions import *

## i. Read the case, department, and source data into their own spark dataframes.

In [2]:
# creating the spark object that activates the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [9]:
# saving file paths to var to call later
case_file = 'csv_files/case.csv'
dept_file = 'csv_files/dept.csv'
source_file = 'csv_files/source.csv'

In [17]:
# reading case csv, putting together code that will use for function next
spark.read.csv(case_file, header = True)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000000|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041670000000|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [18]:
# creating function to read csv files
def read_spark(file_name):
    '''
    This function takes in a csv file name and reads the file contents to a 
    spark DataFrame, returning that df.
    '''
    
    return spark.read.csv(file_name, header = True)

In [27]:
# calling read spark function, passing case file data
df_case = read_spark(case_file)

# calling read spark function, passing dept file data
df_dept = read_spark(dept_file)

# calling read spark function, passing source file data
df_source = read_spark(source_file)

## ii. Write the code necessary to store the source data in both csv and json format.
a. Store these as `sources_csv` and `sources_json`<br>
b. Inspect your folder structure. What do you notice?

#### csv files

In [51]:
# write data to a destination using .write property, writing code to put in function next
df_case.write.csv('case_csv', mode = 'overwrite')

In [52]:
# write data to a destination using .write property, writing code to put in function next
df_dept.write.csv('dept_csv', mode = 'overwrite')

In [53]:
# write data to a destination using .write property, writing code to put in function next
df_source.write.csv('source_csv', mode = 'overwrite')

#### json files

In [54]:
# write data to a destination using .write property, writing code to put in function next
df_case.write.json('case_json', mode = 'overwrite')

In [55]:
# write data to a destination using .write property, writing code to put in function next
df_dept.write.json('dept_json', mode = 'overwrite')

### What I notice when inspecting folder structure . . . 
The data appears to have been split and separate files created from that split data. After a google search on why pyspark splits data when caching I learned the following:
>This is the way spark is designed to write out multiple files in parallel. Writing out many files at the same time is faster for big datasets. <br>([Source: Stack Overflow](https://stackoverflow.com/questions/24371259/how-to-make-saveastextfile-not-split-output-into-multiple-file))

## iii. Inspect the data in your dataframes. 
a. Are the data types appropriate? <br>
> All strings

b. Write the code necessary to cast the values to the appropriate types.

#### inspecting case df

In [59]:
# looking at the dtypes for case data
pd.DataFrame(df_case.dtypes, columns = ['col', 'dtype'])

,col,dtype
0,case_id,string
1,case_opened_date,string
2,case_closed_date,string
3,SLA_due_date,string
4,case_late,string
5,num_days_late,string
6,case_closed,string
7,dept_division,string
8,service_request_type,string
9,SLA_days,string


column | dtype | should be . . . 
:-- | :-- | :--
case_id | string | <<
case_opened_date | string | date
case_closed_date | string | date
SLA_due_date | string | date
case_late | string | int
num_days_late | string | int
case_closed | string | bool
dept_division | string | <<
service_request_type | string | <<
SLA_days | string | int
case_status | string | bool
source_id | string | <<
request_address | string | <<
council_district | string | <<

In [85]:
df_case.select('case_late').distinct().show()

Py4JJavaError: An error occurred while calling o360.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 62.0 failed 1 times, most recent failure: Lost task 3.0 in stage 62.0 (TID 142) (10.10.2.146 executor driver): java.io.FileNotFoundException: 
File file:/Users/stephanie/codeup-data-science/spark-exercises/csv_files/case.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apa

In [70]:
# storing format in var
fmt = 'M/d/yy H:mm'

# casting dates
df_case = df_case.withColumn('case_opened_data', to_timestamp('case_opened_date', fmt))\
    .withColumn('case_closed_data', to_timestamp('case_closed_date', fmt))\
    .withColumn('SLA_due_data', to_timestamp('SLA_due_date', fmt))

# verifying dtypes changed
df_case.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'string'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_opened_data', 'timestamp'),
 ('case_closed_data', 'timestamp'),
 ('SLA_due_data', 'timestamp')]

In [ ]:
# casting integers
df_case.withColumn('case_')

In [60]:
# looking at the dtypes for dept data
pd.DataFrame(df_dept.dtypes, columns = ['col', 'dtype'])

,col,dtype
0,dept_division,string
1,dept_name,string
2,standardized_dept_name,string
3,dept_subject_to_SLA,string


In [61]:
# looking at the dtypes for case data
pd.DataFrame(df_source.dtypes, columns = ['col', 'dtype'])

,col,dtype
0,source_id,string
1,source_username,string


<hr>

# Questions

### 1a. How old is the latest (*in terms of days past SLA*) currently open issue? 
### 1b. How long has the oldest (*in terms of days since opened*) currently opened issue been open? 

### 2. How many Stray Animal cases are there?

### 3. How many service requests that are assigned to the Field Operations department `dept_division` are not classified as "Officer Standby" request type `service_request_type`?

### 4. Convert the `council_district` column to a string column.

### 5. Extract the year from the `case_closed_date` column.

### 6. Convert `num_days_late` from days to hours in new columns `num_hours_late`.

### 7. Join the case data with the source and department data.

### 8. Are there any cases that do not have a request source?

### 9. What are the top 10 service request types in terms of number of requests?

### 10. What are the top 10 service request types in terms of average days late?

### 11. Does number of days late depend on department?

### 12. How do number of days late depend on department and request type?

### max date
You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.